In [34]:
import re

import psycopg2
import pandas as pd

import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

In [43]:
osm_file = '../osm/strava.osm'

In [44]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [45]:
try:
    query = """
        WITH sg_run AS (
          SELECT
            ST_SimplifyPreserveTopology(ST_Transform(location::geometry, 3857), 10) AS location,
            filename
          FROM 
            gpx_points
          WHERE ST_Within( -- within Singapore
            location::geometry,
            ST_MakeEnvelope(
                103.59, -- min lon
                1.13,   -- min lat
                104.04, -- max lon
                1.47,   -- max lat
                4326
            )
          )
        )

        SELECT DISTINCT
          filename,
          ST_Y(ST_Transform(ST_SnapToGrid(ST_Transform(location::geometry, 3857), 10, 10), 4326)) AS lat,
          ST_X(ST_Transform(ST_SnapToGrid(ST_Transform(location::geometry, 3857), 10, 10), 4326)) AS lon
        FROM
          sg_run a
    """

    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()

    df = pd.DataFrame(rows, columns=[
        "filename",
        "lat", 
        "lon",
    ])
    
except Exception as e:
    print(e)
    conn.rollback()

In [46]:
df.head()

,filename,lat,lon
0,11398719881.gpx,1.298583,103.873364
1,11398719881.gpx,1.298583,103.873454
2,11398719881.gpx,1.298673,103.873274
3,11398719881.gpx,1.298673,103.873364
4,11398719881.gpx,1.298673,103.873454


In [47]:
df['node_id'] = df.groupby(['lat', 'lon']).ngroup() + 1

In [48]:
osm = ET.Element("osm")
osm.set("version", "0.6")
osm.set("generator", "CGImap 0.9.2 (596732 spike-08.openstreetmap.org)")
osm.set("copyright", "OpenStreetMap and contributors")
osm.set("attribution", "http://www.openstreetmap.org/copyright")
osm.set("license", "http://opendatacommons.org/licenses/odbl/1-0/")

nodes_df = df\
.drop_duplicates(subset=['node_id', 'lat', 'lon'])\
.sort_values(by='node_id')

for idx, row in nodes_df.iterrows():
    node_id = row["node_id"]

    ET.SubElement(
        osm,
        "node",
        id=str(node_id),
        lat=str(row["lat"]),
        lon=str(row["lon"]),
        visible="true",
        version="1",        
    )

tags = [
      {"k": "highway", "v": "footway"},
      {"k": "footway", "v": "sidewalk"}
  ]

ways_df = nodes_df.groupby('filename')['node_id'].agg(list).reset_index()

for idx, row in ways_df.iterrows():
    way_id = int(re.search(r'\d+', row['filename']).group())

    way = ET.SubElement(
        osm,
        "way",
        id=str(way_id),
        visible="true",
        version="1",
    )

    for node_id in row["node_id"]:
        nd = ET.SubElement(way, "nd")
        nd.set("ref", str(node_id))


    for tag_data in tags:
        tag = ET.SubElement(way, "tag")
        tag.set("k", tag_data["k"])
        tag.set("v", tag_data["v"]) 

    xml_str = minidom.parseString(ET.tostring(osm)).toprettyxml(indent="  ")
    
with open(osm_file, "w") as f:
      f.write(xml_str)